In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import time
import os
import math
import numpy as np
import pyke as pk
import pandas as pd
import k2flix
import matplotlib.pyplot as plt

def collect(oid_list,test):
    tpfs, flux, flares = dict(), dict(), dict()
    for oid in oid_list:

        flares[oid] = pd.read_csv('results/{}/{}_flares.csv'.format(test,oid),
                                 usecols=['istart','istop','istart_no_sys_no_thruster','istop_no_sys_no_thruster'])
        flux[oid] = pd.read_csv('results/{}/{}_flux.csv'.format(test,oid))#,
                                #usecols=['flux_gap','time','flux_model'])
    return flux, flares


#---------------------------------------------------------------------------
# set parameters here
cluster = 'M67'
os.chdir('/home/ekaterina/Documents/appaloosa/stars_shortlist/{}'.format(cluster))
oid_list = pd.read_csv('{}_full.txt'.format(cluster),names=['EPIC'])
oid_list = oid_list.EPIC.tolist()
test = 'run_08'
#----------------------------------------------------------------------------

flux, flares = collect(oid_list, test)

In [2]:
l1, l2 = [], []
for oid in oid_list:

    l1.append(len(flares[oid].istart.dropna()))
    l2.append(len(flares[oid].istart_no_sys_no_thruster.dropna()))

c_flares = pd.DataFrame({'before':l1,'after':l2},index=oid_list)
c_flares['difference'] = (c_flares.after - c_flares.before)/c_flares.before
c_flares.describe()

,after,before,difference
count,75.000000,75.000000,75.000000
mean,5.506667,16.440000,-0.669787
std,5.215293,12.826535,0.139900
min,0.000000,5.000000,-1.000000
25%,2.500000,10.000000,-0.773504
50%,4.000000,13.000000,-0.666667
75%,6.000000,16.000000,-0.575188
max,26.000000,78.000000,-0.200000


In [3]:
flares[oid_list[2]].head()

,istart,istop,istart_no_sys_no_thruster,istop_no_sys_no_thruster
0,5,6,1872.0,1875.0
1,738,745,2737.0,2737.0
2,1338,1344,NaN,NaN
3,1872,1875,NaN,NaN
4,2180,2183,NaN,NaN


In [4]:
flux[oid_list[0]].head()

,Unnamed: 0,flux_gap,flux_model,time,tpf_flags,tpf_time,isflare,new_isflare,new_tpf_flags,tpf_flags_wo_systematics,isflare_wo_systematics,isflare_no_sys_no_thruster
0,0,333716.362671,333847.941791,2306.611038,524288,2306.611038,0.0,0.0,524288,524288,0.0,False
1,1,333702.796190,333847.028039,2306.631471,0,2306.631471,0.0,0.0,0,0,0.0,False
2,2,333710.525828,333848.275669,2306.651903,0,2306.651903,0.0,0.0,0,0,0.0,False
3,3,333782.384248,333851.483213,2306.672335,0,2306.672335,0.0,0.0,0,0,0.0,False
4,4,333863.244913,333856.449226,2306.692767,0,2306.692767,0.0,0.0,0,0,0.0,False


In [5]:
def flag_df(flux):
    
    '''
    
    Plug in here any measure of the results that contains information for your interpretation.
    
    Input:
    
    flux - DataFrame that contains all previous processing.
    
    Return:
    
    flags - Table with customized inputs
    
    '''
    index = [1,8192,524288]
    columns = ['>',
               '>_after_thruster_removal',
               '>_after_systematics_removal',
               '>_left_overall',
               '>_coinciding_with_flare_candidates',
               '>_coinciding_with_clean_flare_candidates',
               '>_relative_coinciding_with_flare_candidates',
               '>_relative_coinciding_with_clean_flare_candidates',
              ]
    flags = pd.DataFrame(index=index, columns=columns)
    for flag in index:
        flags[columns[0]].loc[flag] = flux[flux.tpf_flags >= flag].shape[0]
        flags[columns[1]].loc[flag] = flux[flux.new_tpf_flags >= flag].shape[0]
        flags[columns[2]].loc[flag] = flux[flux.tpf_flags_wo_systematics >= flag].shape[0]
        flags[columns[3]].loc[flag] = flux[(flux.tpf_flags_wo_systematics >= flag) & (flux.new_tpf_flags >= flag)].shape[0]
        flags[columns[4]].loc[flag] = flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[5]].loc[flag] = flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]
        flags[columns[6]].loc[flag] = np.around(flux[(flux.isflare == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)
        flags[columns[7]].loc[flag] = np.around(flux[(flux.isflare_no_sys_no_thruster == True) & (flux.tpf_flags >= flag)].shape[0]/flux.shape[0],3)

    return flags.T

In [6]:
for o in oid_list[:2]:
    print('\n',o)
    print(flag_df(flux[o]))


 228682441
                                                  1      8192   524288
>                                                    457    455    201
>_after_thruster_removal                             452    450    197
>_after_systematics_removal                          369    368    156
>_left_overall                                       367    366    155
>_coinciding_with_flare_candidates                     5      5      4
>_coinciding_with_clean_flare_candidates               0      0      0
>_relative_coinciding_with_flare_candidates        0.001  0.001  0.001
>_relative_coinciding_with_clean_flare_candidates      0      0      0

 211410963
                                                  1      8192   524288
>                                                    275    267    201
>_after_thruster_removal                             257    253    191
>_after_systematics_removal                          217    212    156
>_left_overall                                       

In [7]:
dfi = flag_df(flux[oid_list[0]])
for o in oid_list[1:]:
    dfi = dfi.add(flag_df(flux[o]))
df_av = dfi/len(oid_list)
df_av.to_csv('results/{0}/plots/{1}_{0}_average_flare_stats.csv'.format(test,cluster))

In [8]:
df = flag_df(flux[oid_list[0]]).stack()
df2 = flag_df(flux[oid_list[1]]).stack()
summary = pd.DataFrame({oid_list[0]: df})
for o in oid_list[1:]:
    ser = flag_df(flux[o]).stack()
    ser.name = o
    summary = summary.join(ser)
summary.to_csv('results/{0}/plots/{1}_{0}_all_flare_detail.csv'.format(test,cluster))

In [9]:
std = pd.DataFrame({'maximum':summary.max(axis=1),
                    'minimum': summary.min(axis=1),
                    'average':summary.mean(axis=1),
                    'std dev':summary.std(axis=1)})
std.to_csv('results/{0}/plots/{1}_{0}_all_flags_stats.csv'.format(test,cluster))

In [10]:
df_av


,1,8192,524288
>,362.64,355.6,201
>_after_thruster_removal,343.68,338.493,190.947
>_after_systematics_removal,291.227,285.747,156
>_left_overall,282.6,278.453,151.947
>_coinciding_with_flare_candidates,21.8267,18.9733,10.0533
>_coinciding_with_clean_flare_candidates,2.38667,1.52,0
>_relative_coinciding_with_flare_candidates,0.00594667,0.00513333,0.00272
>_relative_coinciding_with_clean_flare_candidates,0.00072,0.00036,0


In [11]:
std

average  maximum  \
>                                                 1       362.640000  655.000   
                                                  8192    355.600000  655.000   
                                                  524288  201.000000  201.000   
>_after_thruster_removal                          1       343.680000  655.000   
                                                  8192    338.493333  655.000   
                                                  524288  190.946667  201.000   
>_after_systematics_removal                       1       291.226667  550.000   
                                                  8192    285.746667  550.000   
                                                  524288  156.000000  156.000   
>_left_overall                                    1       282.600000  550.000   
                                                  8192    278.453333  550.000   
                                                  524288  151.946667  156.000   
>_coinciding_with_flare_candidates                1        21.826667  118.000   
                                                  8192     18.973333  117.000   
                                                  524288   10.053333   49.000   
>_coinciding_with_clean_flare_candidates          1         2.386667   18.000   
                                                  8192      1.520000   16.000   
                                                  524288    0.000000    0.000   
>_relative_coinciding_with_flare_candidates       1         0.005947    0.032   
                                                  8192      0.005133    0.032   
                                                  524288    0.002720    0.013   
>_relative_coinciding_with_clean_flare_candidates 1         0.000720    0.005   
                                                  8192      0.000360    0.004   
                                                  524288    0.000000    0.000   

                                                          minimum    std dev  
>                                                 1         249.0  64.114265  
                                                  8192      247.0  65.141011  
                                                  524288    201.0   0.000000  
>_after_thruster_removal                          1         236.0  65.975253  
                                                  8192      236.0  66.681448  
                                                  524288    152.0   8.282011  
>_after_systematics_removal                       1         195.0  55.984209  
                                                  8192      194.0  56.806474  
                                                  524288    156.0   0.000000  
>_left_overall                                    1         187.0  56.724798  
                                                  8192      187.0  57.217766  
                                                  524288    129.0   4.311811  
>_coinciding_with_flare_candidates                1           1.0  20.775748  
                                                  8192        1.0  20.720126  
                                                  524288      0.0   8.282011  
>_coinciding_with_clean_flare_candidates          1           0.0   3.384757  
                                                  8192        0.0   3.256428  
                                                  524288      0.0   0.000000  
>_relative_coinciding_with_flare_candidates       1           0.0   0.005683  
                                                  8192        0.0   0.005662  
                                                  524288      0.0   0.002239  
>_relative_coinciding_with_clean_flare_candidates 1           0.0   0.001008  
                                                  8192        0.0   0.000880  
                                                  524288      0.0   0.000000